# Imports

In [ ]:
from database.database import SessionLocal, MediumArticle
from sqlalchemy.sql import func
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from database.database import Author
from sqlalchemy import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# DB 
session = SessionLocal()

# Descriptive Statistics

In [ ]:
stats = session.query(
    func.count(MediumArticle.id).label('total_articles'),
    func.avg(MediumArticle.claps).label('avg_claps'),
    func.avg(MediumArticle.comments_count).label('avg_comments'),
    func.avg(MediumArticle.read_time).label('avg_read_time')
).one()

# Print results
print(f"Total Articles: {stats.total_articles}")
print(f"Average Claps: {stats.avg_claps:.2f}")
print(f"Average Comments: {stats.avg_comments:.2f}")
print(f"Average Read Time: {stats.avg_read_time:.2f} minutes")

# Fetch data for distributions
read_times = session.query(MediumArticle.read_time).all()
claps = session.query(MediumArticle.claps).all()

# Convert to pandas Series
read_times = pd.Series([r[0] for r in read_times if r[0] is not None])
claps = pd.Series([c[0] for c in claps if c[0] is not None])

# Plot distributions
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(read_times, kde=True)
plt.title('Distribution of Read Time')
plt.xlabel('Read Time (minutes)')
plt.subplot(1, 2, 2)
sns.histplot(claps, kde=True)
plt.title('Distribution of Claps')
plt.xlabel('Claps')
plt.tight_layout()
plt.show()

# Time Series Analysis

In [ ]:
articles_over_time = session.query(
    func.date_trunc('month', MediumArticle.date_published).label('month'),
    func.count(MediumArticle.id).label('articles_count')
).group_by('month').order_by('month').all()

# Convert to DataFrame
df_articles = pd.DataFrame(articles_over_time, columns=['month', 'articles_count'])

# Plot articles over time
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_articles, x='month', y='articles_count')
plt.title('Number of Articles Published Over Time')
plt.xlabel('Month')
plt.ylabel('Article Count')
plt.xticks(rotation=45)
plt.show()

# Author Analysis

In [ ]:
# Query author stats with author names

author_stats = session.query(
    Author.username,
    func.count(MediumArticle.id).label('articles_count'),
).join(Author, MediumArticle.author_id == Author.id) \
 .group_by(Author.username).all()

# Convert to DataFrame
df_authors = pd.DataFrame(author_stats, columns=['author_name', 'articles_count'])

# Plot top 10 authors by article count
top_authors = df_authors.nlargest(10, 'articles_count')
plt.figure(figsize=(10, 6))
sns.barplot(data=top_authors, x='author_name', y='articles_count')
plt.title('Top 10 Authors by Article Count')
plt.xlabel('Author Name')
plt.ylabel('Article Count')
plt.xticks(rotation=45, ha='right')
plt.show()


# Tag Analysis

In [ ]:

# Convert the query result to a list of tuples
tag_query = [(row[0], row[1], row[2]) for row in session.execute(text("""
    SELECT tag, COUNT(*) as tag_count, AVG(claps) as avg_claps
    FROM medium_articles, UNNEST(tags) as tag
    GROUP BY tag
    ORDER BY tag_count DESC
""")).fetchall()]

# Convert to DataFrame
df_tags = pd.DataFrame(tag_query, columns=['tag', 'tag_count', 'avg_claps'])

# Ensure 'tag' is of string type
df_tags['tag'] = df_tags['tag'].astype(str)

# apply x.split(":'")[1].split("}")[0] to clean the tags
df_tags['tag'] = df_tags['tag'].apply(lambda x: x.split("'")[3].split("}")[0] if isinstance(x, str) else x)

# Plot top 10 tags by frequency
top_tags = df_tags.nlargest(10, 'tag_count')
plt.figure(figsize=(10, 6))
sns.barplot(data=top_tags, x='tag', y='tag_count')
plt.title('Top 10 Tags by Frequency')
plt.xlabel('Tag')
plt.ylabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.show()

# Plot average claps for top tags
plt.figure(figsize=(10, 6))
sns.barplot(data=top_tags, x='tag', y='avg_claps')
plt.title('Average Claps for Top 10 Tags')
plt.xlabel('Tag')
plt.ylabel('Average Claps')
plt.xticks(rotation=45, ha='right')
plt.show()


# Content Analysis

In [ ]:

texts = session.query(MediumArticle.full_article_text).all()
texts = [t[0] for t in texts if t[0]]

# Word frequency analysis
vectorizer = CountVectorizer(stop_words='english', max_features=20)
word_counts = vectorizer.fit_transform(texts)
word_freq = np.array(word_counts.sum(axis=0)).flatten()
words = vectorizer.get_feature_names_out()

# sort the words and frequencies by frequency
sorted_indices = np.argsort(word_freq)[::-1]
words = words[sorted_indices][:20]
word_freq = word_freq[sorted_indices][:20]

# Plot top 20 words
plt.figure(figsize=(10, 6))
sns.barplot(x=words, y=word_freq)
plt.title('Top 20 Words in Articles')
plt.xlabel('Word')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

# Comment Analysis

In [ ]:
comments_per_article = session.query(MediumArticle.id, MediumArticle.comments_count).all()
df_comments = pd.DataFrame(comments_per_article, columns=['id', 'comments_count']).dropna()

# Plot distribution
plt.figure(figsize=(10, 6))
sns.histplot(df_comments['comments_count'], kde=True)
plt.title('Distribution of Comments per Article')
plt.xlabel('Comments Count')
plt.ylabel('Frequency')
plt.show()

In [ ]:
read_time_claps = session.query(MediumArticle.read_time, MediumArticle.claps).all()
df_read_time = pd.DataFrame(read_time_claps, columns=['read_time', 'claps']).dropna()

# correlation analysis
g = sns.JointGrid(data=df_read_time, x='read_time', y='claps', space=0, xlim=(0, 20), ylim=(0, 1000))
g.plot_joint(sns.kdeplot, fill=True, cmap="rocket", clip=((0, 20), (0, 1000)), thresh=0, levels=100)
g.plot_marginals(sns.histplot, kde=True, color="#03051A", bins=50)

In [ ]:
free_vs_paid = session.query(
    MediumArticle.is_free,
    func.avg(MediumArticle.claps).label('avg_claps')
).group_by(MediumArticle.is_free).all()

# Convert to DataFrame
df_free_paid = pd.DataFrame(free_vs_paid, columns=['is_free', 'avg_claps'])

# Plot
plt.figure(figsize=(8, 6))
sns.barplot(data=df_free_paid, x='is_free', y='avg_claps')
plt.title('Average Claps for Free vs Paid Articles')
plt.xlabel('Is Free')
plt.ylabel('Average Claps')
plt.show()

In [ ]:
# Query claps by publisher_type
publisher_stats = session.query(
    MediumArticle.publisher_type,
    func.avg(MediumArticle.claps).label('avg_claps')
).group_by(MediumArticle.publisher_type).all()

# Convert to DataFrame
df_publisher = pd.DataFrame(publisher_stats, columns=['publisher_type', 'avg_claps'])

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(data=df_publisher, x='publisher_type', y='avg_claps')
plt.title('Average Claps by Publisher Type')
plt.xlabel('Publisher Type')
plt.ylabel('Average Claps')
plt.xticks(rotation=45)
plt.show()

In [ ]:

session = SessionLocal()

# Fetch article texts
texts = [t[0] for t in session.query(MediumArticle.full_article_text).all() if t[0]]

# Vectorize texts
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(texts)

# Fit LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Display topics
terms = vectorizer.get_feature_names_out()
for i, topic in enumerate(lda.components_):
    print(f"Topic {i}: {' '.join([terms[j] for j in topic.argsort()[-10:]])}")

In [ ]:
interactions = session.execute(text("""
    SELECT 
        a.author_id as article_author, 
        c.author_id as comment_author, 
        COUNT(*) as interaction_count,
        aa.username as article_author_name,
        ca.username as comment_author_name
    FROM medium_articles a
    JOIN comments c ON a.id = c.article_id
    JOIN authors aa ON a.author_id = aa.id
    JOIN authors ca ON c.author_id = ca.id
    WHERE a.author_id != c.author_id
    GROUP BY a.author_id, c.author_id, aa.username, ca.username
    ORDER BY interaction_count DESC
""")).fetchall()

# Convert to DataFrame
df_interactions = pd.DataFrame(interactions, columns=['article_author', 'comment_author', 'interaction_count', 'article_author_name', 'comment_author_name'])


# reorder columns to article_author, article_author_name, comment_author, comment_author_name, interaction_count
df_interactions = df_interactions[['article_author', 'article_author_name', 'comment_author', 'comment_author_name', 'interaction_count']]

# make pandas use the whole width of the screen


# Display top interactions
print(df_interactions.head())

In [ ]:
# Sentiment analysis
from textblob import TextBlob

full_article_texts = session.query(MediumArticle.full_article_text).all()
full_article_texts = [t[0] for t in full_article_texts if t[0]]

sentiment_subjectivity = [TextBlob(text).sentiment for text in full_article_texts]
sentiments = [s.polarity for s in sentiment_subjectivity]
objectivities = [1 - s.subjectivity for s in sentiment_subjectivity]
sentiments = pd.Series(sentiments)
objectivities = pd.Series(objectivities)

# distribution of sentiments
plt.figure(figsize=(10, 6))
sns.histplot(sentiments, kde=True)
plt.title('Distribution of Sentiments')
plt.xlabel('Sentiment Polarity')
plt.ylabel('Frequency')
plt.xlim(-1, 1)
plt.show()

# distribution of objectivities
plt.figure(figsize=(10, 6))
sns.histplot(objectivities, kde=True)
plt.title('Distribution of Objectivities') 
plt.xlabel('Objectivity')
plt.ylabel('Frequency')
plt.xlim(0, 1)
plt.show()